In [1]:
from datasets import load_dataset

dataset = load_dataset("metaeval/reclor")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [2]:
"labels", set(dataset["train"]["label"]), dataset["train"][0]

('labels',
 {0, 1, 2, 3},
 {'context': "In rheumatoid arthritis, the body' s immune system misfunctions by attacking healthy cells in the joints causing the release of a hormone that in turn causes pain and swelling. This hormone is normally activated only in reaction to injury or infection. A new arthritis medication will contain a protein that inhibits the functioning of the hormone that causes pain and swelling in the joints.",
  'answers': ['Unlike aspirin and other medications that reduce pain and swelling and that are currently available, the new medication would repair existing cell damage that had been caused by rheumatoid arthritis.',
   'A patient treated with the new medication for rheumatoid arthritis could sustain a joint injury without becoming aware of it.',
   'Joint diseases other than rheumatoid arthritis would not be affected by the new medication.',
   "The benefits to rheumatoid arthritis sufferers of the new medication would outweigh the medication's possible harm

In [3]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

In [4]:
def to_chatml(row):
    context = row["context"]
    label = row["label"]
    answers = row["answers"]
    question = row["question"]
    
    answer_keys = list("ABCDEFGH")
    correct_answer = answers[int(label)]
    correct_answer_key = answer_keys[int(label)]

    formatted_answers = "\n".join([
        f"{key}) {answer}"
        for key, answer in zip(answer_keys, answers)
    ])

    chatml = [
        information(
            f"Context:\n\n{context}",
        ),
        thought(
            f"Question: {question}\n\nChoices:\n{formatted_answers}\n\nCorrect answer: {correct_answer_key}) {correct_answer}",
        ),
    ]

    return dict(chatml=chatml)

In [5]:
dataset = dataset.map(to_chatml).remove_columns(set(dataset["train"].column_names) - {"chatml"})

Map:   0%|          | 0/9276 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
from pprint import pprint
pprint(dataset["train"][0]["chatml"])

[{'content': 'Context:\n'
             '\n'
             "In rheumatoid arthritis, the body' s immune system misfunctions "
             'by attacking healthy cells in the joints causing the release of '
             'a hormone that in turn causes pain and swelling. This hormone is '
             'normally activated only in reaction to injury or infection. A '
             'new arthritis medication will contain a protein that inhibits '
             'the functioning of the hormone that causes pain and swelling in '
             'the joints.',
  'name': 'information',
  'role': 'system'},
 {'content': 'Question: The statements above, if true, most strongly support '
             'which one of the following conclusions?\n'
             '\n'
             'Choices:\n'
             'A) Unlike aspirin and other medications that reduce pain and '
             'swelling and that are currently available, the new medication '
             'would repair existing cell damage that had been caused b

In [7]:
dataset.push_to_hub("diwank/reclor-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]